### 추천 시스템

- 하나의 콘텐츠를 선택했을 때 선택된 콘텐츠와 연관된 추천 콘텐츠가 얼마나 사용자의 관심을 끌고 개인에게 맞춘 콘텐츠를 추천했는지는 사용자가 해당 사이트를 더 강하게 신뢰할 수 있도록 하는 중요한 요소입니다.
- 더 많은 데이터가 추천 시스템에 축적되면서 추천이 더욱 정확해지고 다양한 결과를 얻을 수 있는 좋은 선순환 시스템을 구축하는 것이 중요합니다.

- 추천 시스템의 유형
    - 콘텐츠 기반 필터링
    - 협업 필터링
        - 최근접 이웃 협업 필터링
        - 잠재 요인 협업 필터링


#### 콘텐츠 기반 필터링

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

movie_df = pd.read_csv('tmdb_5000_movies.csv')
print(movie_df.shape)
movie_df.head(1)


(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [2]:
movie_df = movie_df[['id','title','genres','vote_average','vote_count','popularity','keywords','overview']]

In [3]:
pd.set_option('max_colwidth',100)

In [4]:
movie_df.head(1)

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...",7.2,11800,150.437577,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp...","In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ..."


In [5]:
from ast import literal_eval
movie_df['genres'] = movie_df['genres'].apply(literal_eval)
movie_df['keywords'] = movie_df['keywords'].apply(literal_eval)

In [6]:
movie_df['genres'] = movie_df['genres'].apply(lambda x: [y['name'] for y in x])
movie_df['keywords'] = movie_df['keywords'].apply(lambda x : [y['name'] for y in x ])
movie_df[['genres','keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


#### 장르 콘텐츠 유사도 측정

- 영화별 장르 유사도를 측정하기 위해서는 장르를 문자열로 변경한 뒤 피처 벡터화한 후 행렬 데이터 값을 코사인 유사도로 비교하는 것입니다.

- genres 칼럼을 기반으로 하는 콘텐츠 기반 필터링 과정
    1. genres 칼럼을 문자열로 변환 후 Count 피처 벡터화 변환
    2. 피처 벡터화 행렬로 변환한 데이터 세트를 코사인 유사도를 통해 비교, 이를 위해 데이터 세트의 레코드별로 타 레코드와 장르에서 코사인 유사도 값을 가지는 객체를 생성
    3. 장르 유사도가 높은 영화 중 평점이 높은 순으로 영화 추천

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

movie_df['gernes_literal'] = movie_df['genres'].apply(lambda x:(' ').join(x))
count_vect = CountVectorizer(min_df=0,ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movie_df['gernes_literal'])
print(genre_mat.shape)

(4803, 276)


In [8]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:1])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]]


In [9]:
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
print(genre_sim_sorted_ind[:1])

[[   0 3494  813 ... 3038 3037 2401]]


- genre_sim_sorted_ind으로 반환된 값은 유사도 값이 아니라 비교 대상 행의 위치 인덱스임으로 0번 레코드의 경우 자신인 0번 레코드를 제외하면 3494번 레코드가 가장 유사도가 높고, 그 다음이 813번 레코드이며, 가장 유사도가 낮은 레코드는 2401번 레코드라는 뜻입니다.

#### 장르 콘텐츠 필터링을 이용한 영화 추천

In [10]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    
    title_movie = df[df['title'] == title_name]
    
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

In [11]:
similar_movie = find_sim_movie(movie_df, genre_sim_sorted_ind, 'The Godfather',10)
similar_movie[['title','vote_average']].sort_values(by='vote_average',ascending=False)

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1847,GoodFellas,8.2
3866,City of God,8.1
883,Catch Me If You Can,7.7
1243,Mean Streets,7.2
4217,Kids,6.8
2640,Things to Do in Denver When You're Dead,6.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
3636,Light Sleeper,5.7
4065,Mi America,0.0


In [12]:
movie_df[['title','vote_average','vote_count']].sort_values(by='vote_average',ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


- 평가 횟수가 적어 평점이 높은 작품을 필터링하기 위해서 IMDB에서는 평가 횟수에 대한 가중치가 부여된 평점 방식 사용
- 가중 평점 = ( v / (v+m)) * R + ( m / ( v + m )) * C
    
    - v : 개별 영화에 평점을 투표한 횟수
    - m : 평점을 부여하기 위한 최소 투표 횟수
    - R : 개별 영화에 대한 평균 평점
    - C : 전체 영화에 대한 평균 평점


In [13]:
# v = movie_df['vote_count']
# R = movie_df['vote_average']
C = movie_df['vote_average'].mean()
m = movie_df['vote_count'].quantile(0.6) # m값이 높을수록 평점 투표 횟구가 많은 영화에 더 많은 가중 평점 부여

print('C :' ,round(C,3))
print('m :',round(m,3))

C : 6.092
m : 370.2


In [38]:
def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )   

movie_df['weighted_vote'] = movie_df.apply(weighted_vote_average, axis=1)

In [41]:
movie_df[['title','vote_average','weighted_vote','vote_count']].sort_values(by='weighted_vote',ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


In [55]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1)
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

similar_movies = find_sim_movie(movie_df, genre_sim_sorted_ind, 'Iron Man',10)
similar_movies[['title', 'vote_average', 'weighted_vote']]


,title,vote_average,weighted_vote
1990,The Empire Strikes Back,8.2,8.075133
2912,Star Wars,8.1,7.993727
94,Guardians of the Galaxy,7.9,7.833817
1490,Return of the Jedi,7.9,7.767084
183,The Hunger Games: Catching Fire,7.4,7.329476
101,X-Men: First Class,7.1,7.032790
229,Star Wars: Episode III - Revenge of the Sith,7.1,7.016834
51,Pacific Rim,6.7,6.656427
79,Iron Man 2,6.6,6.573959
228,Oblivion,6.4,6.377959
